In [264]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import csv

import re
from collections import Counter
from PIL import Image

from reviews_scraping import get_restaurant_reviews

%matplotlib inline

restaurants_data = pd.read_csv('datasets/TA_restaurants_curated.csv', encoding='utf8', index_col=0)

restaurants_data.head()

,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",/Restaurant_Review-g188590-d11752080-Reviews-M...,d11752080
1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",/Restaurant_Review-g188590-d693419-Reviews-De_...,d693419
2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567.0,"[['Satisfaction', 'Delicious old school restau...",/Restaurant_Review-g188590-d696959-Reviews-La_...,d696959
3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,$$$$,564.0,"[['True five star dinner', 'A superb evening o...",/Restaurant_Review-g188590-d1239229-Reviews-Vi...,d1239229
4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,$$$$,316.0,"[['Best meal.... EVER', 'super food experience...",/Restaurant_Review-g188590-d6864170-Reviews-Li...,d6864170


In [265]:
restaurants_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125527 entries, 0 to 1666
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Name               125527 non-null  object 
 1   City               125527 non-null  object 
 2   Cuisine Style      94176 non-null   object 
 3   Ranking            115876 non-null  float64
 4   Rating             115897 non-null  float64
 5   Price Range        77672 non-null   object 
 6   Number of Reviews  108183 non-null  float64
 7   Reviews            115911 non-null  object 
 8   URL_TA             125527 non-null  object 
 9   ID_TA              125527 non-null  object 
dtypes: float64(3), object(7)
memory usage: 10.5+ MB


# Data 
- Name: name of the restaurant
- City: city location of the restaurant
- Cuisine Style: cuisine style(s) of the restaurant, in a Python list object (94 046 non-null)
    - French, Dutch, European, Vegetarian Friendly, etc.
- Ranking: rank of the restaurant among the total number of restaurants in the city as a float object (115 645 non-null)
    - Float type
- Rating: rate of the restaurant on a scale from 1 to 5, as a float object (115 658 non-null)
    - Float between -1 and 5
- Price Range: price range of the restaurant among 3 categories , as a categorical type (77 555 non-null)
    - $, $$, $$$, $$$$, $-$$, etc.
- Number of Reviews: number of reviews that customers have let to the restaurant, as a float object (108 020 non-null)
    - Float object
- Reviews: 2 reviews that are displayed on the restaurants scrolling page of the city, as a list of list object where the first list contains the 2 reviews, and the second le dates when these reviews were written (115 673 non-null)
    - Python List of two objects
- URL_TA: part of the URL of the detailed restaurant page that comes after 'www.tripadvisor.com' as a string object (124 995 non-null)
- ID_TA: identification of the restaurant in the TA database constructed a one letter and a number (124 995 non-null)

# Data Cleaning

## Turn ranking into categorical datatype 

In [266]:
restaurants_data['Ranking'] = restaurants_data['Ranking'].astype('category')

## Turn Reviews from float to int

In [267]:
restaurants_data['Number of Reviews'] = restaurants_data['Number of Reviews'].fillna(0)
restaurants_data['Number of Reviews'] = restaurants_data['Number of Reviews'].round(0).astype('int')

## Remove duplicated rows by ID_TA
    - There are duplicated rows by ID_TA (125 527 entries and 201 duplicated values)
    - We will remove the duplicated values and keep only the first ones

In [268]:
print(restaurants_data[restaurants_data.ID_TA.duplicated() == True].ID_TA.count())

201


In [269]:
restaurants_data = restaurants_data.drop_duplicates('ID_TA', keep='first')
restaurants_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125326 entries, 0 to 1666
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   Name               125326 non-null  object  
 1   City               125326 non-null  object  
 2   Cuisine Style      94047 non-null   object  
 3   Ranking            115710 non-null  category
 4   Rating             115734 non-null  float64 
 5   Price Range        77574 non-null   object  
 6   Number of Reviews  125326 non-null  int64   
 7   Reviews            115745 non-null  object  
 8   URL_TA             125326 non-null  object  
 9   ID_TA              125326 non-null  object  
dtypes: category(1), float64(1), int64(1), object(7)
memory usage: 10.4+ MB


In [270]:
print(restaurants_data[restaurants_data.ID_TA.duplicated() == True].ID_TA.count())

0


## Rename columns removing blank spaces and capital letters

In [271]:
restaurants_data.rename(columns={'Name': 'name',
            'City': 'city',
            'Cuisine Style':'cuisine_style',
            'Ranking': 'ranking',
            'Rating': 'rating',
            'Price Range':'price_range',
            'Number of Reviews':'reviews_number',
            'Reviews': 'reviews',
            'URL_TA': 'url_ta',
            'ID_TA': 'id_ta'}, inplace=True)

## Replace reviews with rating == -1.0 and setting them to 0
    - There are some reviews bellow 0, which is impossible

In [272]:
print(restaurants_data[restaurants_data.rating == -1.0].city.count())

41


In [273]:
restaurants_data.rating.replace(-1, 0, inplace=True)
print(restaurants_data[restaurants_data.rating == -1.0].city.count())

0


## Eliminate Null values in cuisine_style
    - There are several values of cuisine style null and we decided that these restaurants didn't have enough information, so we will remove them from the dataset.

In [274]:
restaurants_data = restaurants_data[restaurants_data.cuisine_style.isnull()== False]
print(restaurants_data.name.count())
restaurants_data.info()

94047
<class 'pandas.core.frame.DataFrame'>
Int64Index: 94047 entries, 0 to 1666
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   name            94047 non-null  object  
 1   city            94047 non-null  object  
 2   cuisine_style   94047 non-null  object  
 3   ranking         88947 non-null  category
 4   rating          88933 non-null  float64 
 5   price_range     77574 non-null  object  
 6   reviews_number  94047 non-null  int64   
 7   reviews         88942 non-null  object  
 8   url_ta          94047 non-null  object  
 9   id_ta           94047 non-null  object  
dtypes: category(1), float64(1), int64(1), object(7)
memory usage: 8.0+ MB


## Eliminate Null values in ranking
    - There are several values of ranking null and we decided that these restaurants didn't have enough information, so we will remove them from the dataset.

In [275]:
restaurants_data = restaurants_data[restaurants_data.ranking.isnull()== False]
print(restaurants_data.name.count())
restaurants_data.info()

88947
<class 'pandas.core.frame.DataFrame'>
Int64Index: 88947 entries, 0 to 1596
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   name            88947 non-null  object  
 1   city            88947 non-null  object  
 2   cuisine_style   88947 non-null  object  
 3   ranking         88947 non-null  category
 4   rating          88853 non-null  float64 
 5   price_range     75406 non-null  object  
 6   reviews_number  88947 non-null  int64   
 7   reviews         88852 non-null  object  
 8   url_ta          88947 non-null  object  
 9   id_ta           88947 non-null  object  
dtypes: category(1), float64(1), int64(1), object(7)
memory usage: 7.6+ MB


## Eliminate Null values in rating
    - There are several values of rating null and we decided that these restaurants didn't have enough information, so we will remove them from the dataset.

In [276]:
restaurants_data = restaurants_data[restaurants_data.rating.isnull()== False]
print(restaurants_data.name.count())
restaurants_data.info()

88853
<class 'pandas.core.frame.DataFrame'>
Int64Index: 88853 entries, 0 to 1593
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   name            88853 non-null  object  
 1   city            88853 non-null  object  
 2   cuisine_style   88853 non-null  object  
 3   ranking         88853 non-null  category
 4   rating          88853 non-null  float64 
 5   price_range     75339 non-null  object  
 6   reviews_number  88853 non-null  int64   
 7   reviews         88852 non-null  object  
 8   url_ta          88853 non-null  object  
 9   id_ta           88853 non-null  object  
dtypes: category(1), float64(1), int64(1), object(7)
memory usage: 7.6+ MB


## Eliminate Null values in price_range
    - There are several values of price range null and we decided that these restaurants didn't have enough information, so we will remove them from the dataset.

In [277]:
restaurants_data = restaurants_data[restaurants_data.price_range.isnull()== False]
print(restaurants_data.name.count())
restaurants_data.info()

75339
<class 'pandas.core.frame.DataFrame'>
Int64Index: 75339 entries, 0 to 1593
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   name            75339 non-null  object  
 1   city            75339 non-null  object  
 2   cuisine_style   75339 non-null  object  
 3   ranking         75339 non-null  category
 4   rating          75339 non-null  float64 
 5   price_range     75339 non-null  object  
 6   reviews_number  75339 non-null  int64   
 7   reviews         75338 non-null  object  
 8   url_ta          75339 non-null  object  
 9   id_ta           75339 non-null  object  
dtypes: category(1), float64(1), int64(1), object(7)
memory usage: 6.5+ MB


## Eliminate Null values in reviews
    - There are several values of reviews that are null and we decided that these restaurants didn't have enough information, so we will remove them from the dataset.

In [278]:
restaurants_data = restaurants_data[restaurants_data.reviews.isnull()== False]
print(restaurants_data.name.count())
restaurants_data.info()

75338
<class 'pandas.core.frame.DataFrame'>
Int64Index: 75338 entries, 0 to 1593
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   name            75338 non-null  object  
 1   city            75338 non-null  object  
 2   cuisine_style   75338 non-null  object  
 3   ranking         75338 non-null  category
 4   rating          75338 non-null  float64 
 5   price_range     75338 non-null  object  
 6   reviews_number  75338 non-null  int64   
 7   reviews         75338 non-null  object  
 8   url_ta          75338 non-null  object  
 9   id_ta           75338 non-null  object  
dtypes: category(1), float64(1), int64(1), object(7)
memory usage: 6.5+ MB


## Normalize Review links
    - It is missing the "www.tripadvisor.com" from the URL_TA column

In [279]:
restaurants_data["url_ta"] = "https://www.tripadvisor.com" + restaurants_data["url_ta"]
restaurants_data.head()

,name,city,cuisine_style,ranking,rating,price_range,reviews_number,reviews,url_ta,id_ta
0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136,"[['Just like home', 'A Warm Welcome to Wintry ...",https://www.tripadvisor.com/Restaurant_Review-...,d11752080
1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812,"[['Great food and staff', 'just perfect'], ['0...",https://www.tripadvisor.com/Restaurant_Review-...,d693419
2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567,"[['Satisfaction', 'Delicious old school restau...",https://www.tripadvisor.com/Restaurant_Review-...,d696959
3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,$$$$,564,"[['True five star dinner', 'A superb evening o...",https://www.tripadvisor.com/Restaurant_Review-...,d1239229
4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,$$$$,316,"[['Best meal.... EVER', 'super food experience...",https://www.tripadvisor.com/Restaurant_Review-...,d6864170


# Create a new csv file for reviews
    - Separate the reviews from each restaurant into a different table

In [280]:
count = 0

for index, row in restaurants_data.iterrows():
    aux_reviews = row.reviews.split("'], ")[0]
    aux_dates = row.reviews.split('], ')[1]

    final_reviews = aux_reviews.split("', '")
    if len(final_reviews) == 1:
        final_reviews = aux_reviews.split('",')
    final_dates = aux_dates.split("', '")

    if len(final_dates) != len(final_reviews):
        print("Reviews size and dates don't match - restaurant " + str(row.id_ta))
        print("Reviews")
        for i in range(len(final_reviews)):
            print(final_reviews[i])
        print("Dates")
        for j in range(len(final_dates)):
            print(final_dates[j])
        count = count + 1
        print("Total number of errors: " + str(count))

    

Reviews size and dates don't match - restaurant d3589386
Reviews
[[nan, 'Good food with decent service
Dates
['11/21/2017
07/21/2017']]
Total number of errors: 1
Reviews size and dates don't match - restaurant d10278680
Reviews
[['Lovely, cosy restaurant, pleasant staff, d...
Memorable dinner [Nov.], breakfast should...
Dates
breakfast should...'
Total number of errors: 2
Reviews size and dates don't match - restaurant d8811565
Reviews
[['A very good "Francesinha
 indeed
Dates
['03/30/2017']]
Total number of errors: 3
Reviews size and dates don't match - restaurant d959060
Reviews
[['Good food, attentive staff
Sternstraße 125, Schanzenviertel, [59], ju...
Dates
ju...'
Total number of errors: 4


## Errors
- After searching through all reviews, we found out that 4 restaurants had bad structured reviews, so we decided to eliminate them.
- Errors occurred on restaurants:
    - d3589386
    - d10278680
    - d8811565
    - d959060

In [281]:
restaurants_data = restaurants_data[restaurants_data.id_ta != "d3589386"]
restaurants_data = restaurants_data[restaurants_data.id_ta != "d10278680"]
restaurants_data = restaurants_data[restaurants_data.id_ta != "d8811565"]
restaurants_data = restaurants_data[restaurants_data.id_ta != "d959060"]

restaurants_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75334 entries, 0 to 1593
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   name            75334 non-null  object  
 1   city            75334 non-null  object  
 2   cuisine_style   75334 non-null  object  
 3   ranking         75334 non-null  category
 4   rating          75334 non-null  float64 
 5   price_range     75334 non-null  object  
 6   reviews_number  75334 non-null  int64   
 7   reviews         75334 non-null  object  
 8   url_ta          75334 non-null  object  
 9   id_ta           75334 non-null  object  
dtypes: category(1), float64(1), int64(1), object(7)
memory usage: 6.5+ MB


In [282]:
path_to_file = "datasets/reviews_table.csv"

csvFile = open(path_to_file, 'a', encoding="utf-8")
csvWriter = csv.writer(csvFile)

for index, row in restaurants_data.iterrows():
    aux_reviews = row.reviews.split("'], ")[0]
    aux_dates = row.reviews.split('], ')[1]

    final_reviews = aux_reviews.split("', '")
    if len(final_reviews) == 1:
        final_reviews = aux_reviews.split('",')
    final_dates = aux_dates.split("', '")

    for i in range(len(final_reviews)):
        review = final_reviews[i].replace("[", "")
        review = review.replace("'", "")
        review = review.replace("]", "")
        review = review.replace('"', '')
        final_review = review.strip('"')

        date = final_dates[i].replace("[", "")
        date = date.replace("]", "")
        date = date.replace("'", "")
        date = date.replace('"', "")

        if len(final_review) > 1:
            csvWriter.writerow([row.id_ta, final_review, date]) 

# Create a new csv file for cuisine styles
    - Separate the cuisine styles from each restaurant into a different table

In [283]:
path_to_cuisine_file = "datasets/cuisine_style_table.csv"

csv_cuisine_file = open(path_to_cuisine_file, 'a', encoding="utf-8")
csv_cuisine_writer = csv.writer(csv_cuisine_file)

for index, row in restaurants_data.iterrows():
    aux_styles = row.cuisine_style.strip("[]'")
    aux_styles = aux_styles.split(", ")

    for i in range(len(aux_styles)):
        style = aux_styles[i]
        style = style.strip("'")
        csv_cuisine_writer.writerow([row.id_ta, style]) 

# Copy of the clean dataset

In [284]:
restaurants = restaurants_data.copy()

path_to_clean_file = "datasets/clean_ta_dataset.csv"

csv_clean_file = open(path_to_clean_file, 'a', encoding="utf-8")
csv_clean_writer = csv.writer(csv_clean_file)

for index, row in restaurants_data.iterrows():
    csv_clean_writer.writerow(row) 


In [285]:
restaurants_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75334 entries, 0 to 1593
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   name            75334 non-null  object  
 1   city            75334 non-null  object  
 2   cuisine_style   75334 non-null  object  
 3   ranking         75334 non-null  category
 4   rating          75334 non-null  float64 
 5   price_range     75334 non-null  object  
 6   reviews_number  75334 non-null  int64   
 7   reviews         75334 non-null  object  
 8   url_ta          75334 non-null  object  
 9   id_ta           75334 non-null  object  
dtypes: category(1), float64(1), int64(1), object(7)
memory usage: 6.5+ MB


# Web scraping the reviews of each one of the restaurants

In [286]:
# # generate random integer values
# from random import seed
# from random import randint

# clean_data_scraper = pd.read_csv('datasets/clean_ta_dataset.csv', encoding='utf8', index_col=0)

# # seed random number generator
# seed(1)

# count = 0

# next_value = 0

# for index, row in clean_data_scraper.iterrows():
#     if(count > next_value):
#         next_value = next_value + randint(0, 200)
#         print('Getting reviews from ' + str(count))
#         get_restaurant_reviews(row.id_ta, row.url_ta)
#     count = count + 1

# Data after scraping

In [386]:
reviews_scraped = pd.read_csv('datasets/scraping/reviews.csv')
print(reviews_scraped['restaurant_id'].nunique())
reviews_scraped.info()


2123
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28890 entries, 0 to 28889
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   restaurant_id  28890 non-null  object
 1   date           28729 non-null  object
 2   rating         28890 non-null  int64 
 3   title          28890 non-null  object
 4   review         28890 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.1+ MB


# Merge everything in a final csv

## Merge every review in one dataset
- Eliminate the column of reviews that aren't cleaned.
- Append all the reviews for each restaurant, with the correct format.

## Final csv
- Copy everything to the final csv.

In [387]:
cleaned_dataset = pd.read_csv('datasets/clean_ta_dataset.csv')

initial_reviews = pd.read_csv('datasets/reviews_table.csv')
scraper_reviews = pd.read_csv('datasets/scraping/reviews.csv')

cuisine_styles = pd.read_csv('datasets/cuisine_style_table.csv')

total_reviews = {}

total_cuisine_styles = {}

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

# Getting reviews from scraper_reviews
for index_scraper, row_scraper in scraper_reviews.iterrows():
    if row_scraper.restaurant_id in total_reviews:
        total_reviews[row_scraper.restaurant_id].append(emoji_pattern.sub(r'', row_scraper.review))
    else:
        total_reviews[row_scraper.restaurant_id] = [emoji_pattern.sub(r'', row_scraper.review)]

# Getting reviews from initial_reviews
for index_initial, row_initial in initial_reviews.iterrows():
    if not row_initial.review == ', ': # Some reviews are empty
        if row_initial.restaurant_id in total_reviews:
            total_reviews[row_initial.restaurant_id].append(row_initial.review)
        else:
            total_reviews[row_initial.restaurant_id] = [row_initial.review]

# Getting cuisine styles from cuisine_styles
for index_cuisine, row_cuisine in cuisine_styles.iterrows():
    if row_cuisine.restaurant_id in total_cuisine_styles:
        total_cuisine_styles[row_cuisine.restaurant_id].append(row_cuisine.cuisine_style)
    else:
        total_cuisine_styles[row_cuisine.restaurant_id] = [row_cuisine.cuisine_style]

for index, row in cleaned_dataset.iterrows():
    if row.restaurant_id in total_reviews:
        cleaned_dataset.at[index, 'reviews'] = total_reviews[row.restaurant_id]
    else:
         cleaned_dataset.at[index, 'reviews'] = None
    
    if row.restaurant_id in total_cuisine_styles:
        cleaned_dataset.at[index, 'cuisine_style'] = total_cuisine_styles[row.restaurant_id]
    else:
         cleaned_dataset.at[index, 'cuisine_style'] = None


In [388]:
cleaned_dataset = cleaned_dataset[cleaned_dataset.reviews.isnull()== False]
cleaned_dataset = cleaned_dataset[cleaned_dataset.cuisine_style.isnull()== False]

cleaned_dataset['reviews'] = cleaned_dataset['reviews'].values.tolist()
cleaned_dataset['cuisine_style'] = cleaned_dataset['cuisine_style'].values.tolist()

cleaned_dataset.to_csv('datasets/final_data.csv')

# Convert to json

In [389]:
import json

restaurant_dict = cleaned_dataset.to_dict(orient='records')

with open("datasets/json/restaurants.json", "w") as f:
    f.write(json.dumps(restaurant_dict, indent=4))
    f.close()